In [1]:
import matplotlib.pyplot as plt

import re, os, time, random
import pandas as pd
import numpy as np

import tensorflow as tf
import pickle as pkl

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.models import Model, load_model
from keras.layers import LSTM, GRU, Input, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences

from nltk.translate.bleu_score import sentence_bleu

In [40]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [2]:
SEED = 1337

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 1. Подготовка даных

In [3]:
def preprocess_sentence(w, punctuation=False, toSeq=False):
    """
        Функция для предобработки 
    """
    
    # Уменьшаем регистр и убираем лишние пробелы
    w = w.lower().strip()
    
    # Замена всех символов 'æ' на однотипный
    w = re.sub(r"ӕ", r"æ", w)
    
    # Удаление апострофом
    w = re.sub("'", '', w)
    
    if punctuation:
        # Делаем между словом и знаком пунктуации отступ 'слово! -> слово !'
        w = re.sub(r"([?.!,])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)
    else:
        # Удаляет все знаки пунктуации
        w = re.sub(r"[^\w\s]", r"", w)
    
    # Выкидываем все остальные символы из рассмотрения 
    w = re.sub(r"[^a-яА-Яa-zA-Z?.!,æё]+", " ", w)
    w = w.rstrip().strip()
    
    # Добавляем токены для начала и конца предложения
    if toSeq:
        w = f'<sos> {w} <eos>'
        
    return w

In [4]:
def load_dataset(path):
    """
        Функция, которая создаёт датасет
    """    
    new_path = f'Date/{path}'
    with open(new_path, encoding='utf-8') as f:
        data = f.read()
    
    uncleaned_data_list = data.split('\n')
    
    source_word = []
    target_word = []
    for word in uncleaned_data_list:
        source_word.append(preprocess_sentence(word.split('\t')[0], punctuation=False, toSeq=False))
        target_word.append(preprocess_sentence(word.split('\t')[1], punctuation=False, toSeq=True))
        
    language_data = pd.DataFrame(columns=['Source','Target'])
    language_data['Source'] = source_word
    language_data['Target'] = target_word
    
    return language_data

In [5]:
def tokenizer_(text_data):
    '''
        Токенайзер
    '''
    
    tokenizer = Tokenizer(filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
    tokenizer.fit_on_texts(text_data)
    return tokenizer

In [6]:
def max_length(data):
    max_length_ = max([len(x.split(' ')) for x in data])
    return max_length_

In [7]:
def get_preparing_data(input_seq, output_seq):
    tokenizer_input, tokenizer_output = tokenizer_(input_seq), tokenizer_(output_seq)
    input_max_length, output_max_length = len(tokenizer_input.word_index) + 1, len(tokenizer_output.word_index) + 1
    
    return tokenizer_input, tokenizer_output, input_max_length, output_max_length

# 2. Построение модели Encoder-Decoder

In [8]:
class Encoder(tf.keras.Model):
    """
        Энкодер 
    """
    
    def __init__(self, vocab_size_input, HIDDEN_DIM):
        super(Encoder, self).__init__()
        
        self.inputs = Input(shape=(None,), name="encoder_inputs")
        self.embedding = Embedding(vocab_size_input, HIDDEN_DIM, mask_zero=True, name="encoder_embedding")(self.inputs)
        
        encoder = LSTM(HIDDEN_DIM, return_state=True, name="encoder_lstm")
        self.outputs, state_h, state_c = encoder(self.embedding)
        self.states = [state_h, state_c]
        
def getEncoder(model_loaded):
    encoder_inputs_inf = model_loaded.input[0]
    encoder_outputs_inf, inf_state_h, inf_state_c = model_loaded.layers[4].output
    encoder_inf_states = [inf_state_h,inf_state_c]

    return Model(encoder_inputs_inf, encoder_inf_states, name='Encoder') 

In [9]:
class Decoder(tf.keras.Model):
    """
        Декодер 
    """
    
    def __init__(self, vocab_size_output, HIDDEN_DIM, encoder_states):
        super(Decoder, self).__init__()
        
        self.inputs = Input(shape=(None,), name="decoder_inputs")
        self.embedding = Embedding(vocab_size_output, HIDDEN_DIM, mask_zero=True, name="decoder_embedding")(self.inputs)
        
        decoder = LSTM(HIDDEN_DIM, return_sequences=True, return_state=True, name="decoder_lstm")
        self.outputs, _, _ = decoder(self.embedding, initial_state=encoder_states)
        self.dense = Dense(vocab_size_output, activation='softmax', name="dense_lstm")
        self.outputs = self.dense(self.outputs)
        
def getDecoder(model_loaded):
    decoder_state_h_input = Input(shape=(HIDDEN_DIM,))
    decoder_state_c_input = Input(shape=(HIDDEN_DIM,))
    decoder_state_input = [decoder_state_h_input,decoder_state_c_input]

    decoder_input_inf = model_loaded.input[1]
    decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
    decoder_lstm_inf = model_loaded.layers[5]
    decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state=decoder_state_input)
    decoder_state_inf = [decoder_state_h_inf, decoder_state_c_inf]
    dense_inf = model_loaded.layers[6]
    decoder_output_final = dense_inf(decoder_output_inf)

    return Model([decoder_input_inf]+decoder_state_input, [decoder_output_final]+decoder_state_inf, name='Decoder')

In [10]:
def Batch(X, Y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_data_input = np.zeros((batch_size,max_lenght_source),dtype='float32') #metrix of batch_size*max_length_english
            decoder_data_input = np.zeros((batch_size,max_lenght_target),dtype='float32') #metrix of batch_size*max_length_marathi
            decoder_target_input = np.zeros((batch_size,max_lenght_target,vocab_size_target),dtype='float32') # 3d array one hot encoder decoder target data
            for i, (input_text,target_text) in enumerate(zip(X[j:j+batch_size],Y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_data_input[i,t] = tokenizer_input.word_index[word] # Here we are storing the encoder 
                                                                         #seq in row here padding is done automaticaly as 
                                                                         #we have defined col as max_lenght
                for t, word in enumerate(target_text.split()):
                    decoder_data_input[i,t] = tokenizer_output.word_index[word] # same for the decoder sequence
                    if t>0:
                        decoder_target_input[i,t-1,tokenizer_output.word_index[word]] = 1 #target is one timestep ahead of decoder input because it does not have 'start tag'
            yield ([encoder_data_input,decoder_data_input],decoder_target_input)

# 3. Входные параметры

In [11]:
HIDDEN_DIM = 256
batch_size = 6
epochs = 20

start_target = "<sos>"
end_target = "<eos>"

In [12]:
path = 'rus-oss.txt'
data = load_dataset(path)
input_seq, output_seq = data['Source'].values, data['Target'].values
tokenizer_input, tokenizer_output, vocab_size_source, vocab_size_target = get_preparing_data(input_seq, output_seq)

In [13]:
with open(f'{path[:-4]}-tokenizer_input.pkl','wb') as f:
    pkl.dump(tokenizer_input, f)

with open(f'{path[:-4]}-tokenizer_output.pkl','wb') as f:
    pkl.dump(tokenizer_output, f)

pkl.dump(tokenizer_input, open(f'{path[:-4]}-tokenizer_input.pkl', 'wb'))
pkl.dump(tokenizer_output, open(f'{path[:-4]}-tokenizer_output.pkl', 'wb'))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(input_seq, output_seq, test_size = 0.1)
train_samples = len(X_train)
test_samples = len(X_test)

max_lenght_source = max_length(X_train)
max_lenght_target = max_length(y_train)

In [15]:
encoder = Encoder(vocab_size_source, HIDDEN_DIM)
decoder = Decoder(vocab_size_target, HIDDEN_DIM, encoder.states)

model = Model([encoder.inputs, decoder.inputs], decoder.outputs, name="LSTM-Translation")

In [16]:
loss_function = [
    'categorical_crossentropy',
    'binary_crossentropy',
]

now_loss_function = loss_function[0]
model.compile(loss=now_loss_function, optimizer='rmsprop', metrics=['accuracy'])

# 4. Обучение модели

In [17]:
# %%capture
model.fit_generator(generator = Batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size, epochs=epochs)

Epoch 1/20


C:\Users\ONIGAT~1\AppData\Local\Temp/ipykernel_7040/1321299230.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = Batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size, epochs=epochs)


70/70 [==============================] - 10s 54ms/step - loss: 1.2815 - accuracy: 0.1566
Epoch 2/20
70/70 [==============================] - 4s 57ms/step - loss: 1.1754 - accuracy: 0.1690
Epoch 3/20
70/70 [==============================] - 4s 53ms/step - loss: 1.1216 - accuracy: 0.1711
Epoch 4/20
70/70 [==============================] - 4s 52ms/step - loss: 1.0644 - accuracy: 0.1735
Epoch 5/20
70/70 [==============================] - 4s 51ms/step - loss: 1.0283 - accuracy: 0.1767
Epoch 6/20
70/70 [==============================] - 4s 53ms/step - loss: 0.9812 - accuracy: 0.1790
Epoch 7/20
70/70 [==============================] - 4s 51ms/step - loss: 0.9324 - accuracy: 0.1868
Epoch 8/20
70/70 [==============================] - 4s 52ms/step - loss: 0.8906 - accuracy: 0.1952
Epoch 9/20
70/70 [==============================] - 4s 54ms/step - loss: 0.8513 - accuracy: 0.2051
Epoch 10/20
70/70 [==============================] - 4s 56ms/step - loss: 0.8114 - accuracy: 0.2210
Epoch 11/20
70/70 [

# 5. Сохранение модели

In [18]:
# Отрисовка схемы модели
# plot_model(model, to_file=f'{dir}-{epochs}-train_model.png', show_shapes=True)

In [19]:
def model_save_JSON():
    model_json = model.to_json()
    
    with open(f'LSTM-[{path[:-4]}]-[Epochs={epochs}]-[LossFunction={now_loss_function}].json', "w") as json_file:
        json_file.write(model_json)
    
    model.save_weights(f'LSTM-[{path[:-4]}]-[Epochs={epochs}]-[LossFunction={now_loss_function}]-[weight].h5')
    print("Saved model to disk")

In [20]:
model_save_JSON()

Saved model to disk


# 6. Загрузка модели

In [21]:
def model_load_JSON():
    json_file = open(f'LSTM-[{path[:-4]}]-[Epochs={epochs}]-[LossFunction={now_loss_function}].json')
    loaded_model_json = json_file.read()
    json_file.close()
    model_loaded = model_from_json(loaded_model_json)

    model_loaded.load_weights(f'LSTM-[{path[:-4]}]-[Epochs={epochs}]-[LossFunction={now_loss_function}]-[weight].h5')
    print("Model loaded")
    
    return model_loaded

In [22]:
model_loaded = model_load_JSON()

Model loaded


In [23]:
model_loaded.summary()

Model: "LSTM-Translation"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 encoder_embedding (Embedding)  (None, None, 256)    241920      ['encoder_inputs[0][0]']         
                                                                                                  
 decoder_embedding (Embedding)  (None, None, 256)    236032      ['decoder_inputs[0][0]']         
                                                                                   

In [24]:
encoder_model = getEncoder(model_loaded)

In [25]:
decoder_model = getDecoder(model_loaded)

In [26]:
with open(f'{path[:-4]}-tokenizer_input.pkl','rb') as f:
    tokenizer_input = pkl.load(f)
with open(f'{path[:-4]}-tokenizer_output.pkl','rb') as f:
    tokenizer_output = pkl.load(f)

reverse_word_map_input = dict(map(reversed, tokenizer_input.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_output.word_index.items()))

# 7. Проверка

In [27]:
def decode_seq(input_seq):
    state_values_encoder = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tokenizer_output.word_index[start_target]
    stop_condition = False
    decoder_sentance = ''
    
    while not stop_condition:
        sample_word , decoder_h,decoder_c= decoder_model.predict([target_seq] + state_values_encoder)
        sample_word_index = np.argmax(sample_word[0,-1,:])
        decoder_word = reverse_word_map_target[sample_word_index]
        decoder_sentance += ' '+ decoder_word
        if (decoder_word == end_target or 
            len(decoder_sentance) > 70):
            stop_condition = True
        target_seq[0, 0] = sample_word_index
        state_values_encoder = [decoder_h,decoder_c]
    return decoder_sentance

In [28]:
for i in range(10):
    sentance = X_train[i]
    original_target = y_train[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
    predicted_target = decode_seq(pad_sequence)
    reference = original_target[5:-5].strip().split()
    hypothetic = predicted_target[:-5].strip().split()
    
    print("Test:\t", i+1)
    print("  Sentance:\t", sentance)
    print("  Origianl:\t", original_target[5:-5])
    print("  Predicted:\t", predicted_target[:-5])
    print("  BLEU:\t", sentence_bleu([hypothetic], reference, weights = [1]))
    print("=="*50)

Test:	 1
  Sentance:	 здесь не бываю
  Origianl:	  ам нæ вæййын 
  Predicted:	  нæ дæн 
  BLEU:	 0.3333333333333333
Test:	 2
  Sentance:	 хорошо
  Origianl:	  хорз 
  Predicted:	  хорз 
  BLEU:	 1.0
Test:	 3
  Sentance:	 ктото позвонил
  Origianl:	  чидæр æрбадзырдта 
  Predicted:	  камерæ айс 
  BLEU:	 0
Test:	 4
  Sentance:	 кто в этом году идет в школу
  Origianl:	  ацы аз скъоламæ чи цæудзæн 
  Predicted:	  ацы аз скъоламæ скъоламæ 
  BLEU:	 0.6
Test:	 5
  Sentance:	 где ты отдыхал
  Origianl:	  дæ фæллад кæм уагътай 
  Predicted:	  кæм дæ хорзæхæй 
  BLEU:	 0.5
Test:	 6
  Sentance:	 кот
  Origianl:	  гæды 
  Predicted:	  фос 
  BLEU:	 0
Test:	 7
  Sentance:	 у меня нет денег но есть мечты
  Origianl:	  æхца мæм нæй фæлæ мæм ис бæллицтæ 
  Predicted:	  уый дæр дæр дæр дæр ис ахуырадон кълас 
  BLEU:	 0.12383969996431167
Test:	 8
  Sentance:	 счастье вашему дому
  Origianl:	  фарн уæ хæдзары 
  Predicted:	  алфавиты дамгъæ равзар 
  BLEU:	 0
Test:	 9
  Sentance:	 жадный
  Origianl:	

In [29]:
for i in range(10):
    sentance = X_test[i]
    original_target = y_test[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
    predicted_target = decode_seq(pad_sequence)
    reference = original_target[5:-5].strip().split()
    hypothetic = predicted_target[:-5].strip().split()
    
    print("Test:\t", i+1)
    print("  Sentance:\t", sentance)
    print("  Origianl:\t", original_target[5:-5])
    print("  Predicted:\t", predicted_target[:-5])
    print("  BLEU:\t", sentence_bleu([hypothetic], reference, weights = [1]))
    print("=="*50)

Test:	 1
  Sentance:	 нет я работал у меня отпуск будет зимой
  Origianl:	  нæ кусгæ кæнын мæнæн ме нцойуагъд зымæджы уыдзæн 
  Predicted:	  æз дæр дæр дæр дæр нæ федтай 
  BLEU:	 0.12500000000000003
Test:	 2
  Sentance:	 расскажите нам всё что знаете
  Origianl:	  радзурут нын алцыдæр цы зонут 
  Predicted:	  нæ цы æрцыди дыл нæу 
  BLEU:	 0.2
Test:	 3
  Sentance:	 у нас возле дома есть автобусная остановка
  Origianl:	  нæ хæдзары цур ис автобусы æрлæууæн 
  Predicted:	  мæ фыд дыууæ нана уайтагъд ног 
  BLEU:	 0
Test:	 4
  Sentance:	 он его нашёл
  Origianl:	  уый йæ ссардта 
  Predicted:	  уый бон у 
  BLEU:	 0.3333333333333333
Test:	 5
  Sentance:	 мои уши красные
  Origianl:	  мæ хъустæ сырx сты 
  Predicted:	  дæ ном куыд алан 
  BLEU:	 0
Test:	 6
  Sentance:	 том ест корм для кошек
  Origianl:	  том хæры гæдиты хæринаг 
  Predicted:	  том мæнæй том дыл и 
  BLEU:	 0.19470019576785122
Test:	 7
  Sentance:	 все хорошо спасибо
  Origianl:	  бузныг ницы мын у 
  Predicted:	  хорз х